In [1]:
%run Latex_macros.ipynb
%run beautify_plots.py

<IPython.core.display.Latex object>

In [2]:
# My standard magic !  You will see this in almost all my notebooks.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

%matplotlib inline

In [3]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Common imports
import os

import svm_helper
%aimport svm_helper
svmh = svm_helper.SVM_Helper()

%matplotlib inline


# SVM Cost function derived from Constrained Optimization

Here is an alternate (and slightly more mathematical) derivation of the SVM Cost function.

Recall our dual objectives
- Maximize margin (width of buffer zone)
- Minimize classification loss (incorrectly classified or in-buffer training examples)

The natural way to express these objectives is as a Constrained Optimization problem
- Maximize margin
- subject to not violating a boundary constraint (all examples on correct side of margin boundary)

We already showed
 how maximizing the margin was equivalent to minimizing the margin penalty
$$
\frac{1}{2} \Thetam^T \cdot \Thetam
$$

Each per-example Classification Loss
$$
\max{} \left( 0,  1 - \transy^\ip * s(\hat{\x}^\ip ) \right)
$$

is equivalent to an inequality constraint
$$
\begin{array}[lll]\\
 1 - \transy^\ip * s(\hat{\x}^\ip) \le 0 \\
 \transy^\ip * s(\hat{\x}^\ip) \ge 1  & \text{re-arranging the terms} \\
 \end{array}
$$



So we can express the Loss Function minimization as a constrained optimization problem

$$
\begin{array}[llll]\\
\text{minimize } \frac{1}{2} \Thetam^T \Thetam \\
\text{subject to }  \transy^\ip * s(\hat{\x}^\ip) \ge 1 \text{for } i=1,\ldots,m
\end{array}
$$

The objective
$$
\frac{1}{2} \Thetam^T \Thetam
$$
 is quadratic so this is a Quadratic Optimization problem.

## Solving Constrained Optimization: LaGrangian multipliers

It is beyond the scope of this course, but one way to solve constrained minimization
is to create an objective function $\loss$ that is the sum of
- the function to minimize
- $\lambda$ times each constraint (when it is rewritten in a form where the inequality is with respect to $0$)

The $\lambda$ terms are called *Lagrangian multipliers*.

They serve as penalties when a constraint is violated (i.e., is greater than $0$).

The (per example) objective function $\loss$ becomes
$$
 \frac{1}{2} \Thetam^T \Thetam + \lambda * ( \max{}(0, - 1 * \transy^\ip * s(\x^\ip)))
$$

which is equal to the SVM Cost function that we constructed in the ad hoc fashion
if we let $\lambda = C$.

Recall $C$ expressed the relative weight between the Margin Penalty and Classification Loss.

**Aside**

- You will sometimes see the max replaced by a "slack" variable $\xi^\ip$
$$\xi^\ip = \max{}(0, - 1 * \transy^\ip * s(\x^\ip))$$

    This is a way of turning an inequality 
$$
\transy^\ip * s(\hat{\x}^\ip) \ge 1
$$
into an equality
    - i.e., the "slack" is the amount of the difference of the expression from being equal
- Technically, there should be a separate Lagrangian for each constraint
    - should write $\lambda^\ip$ for $i = 1 \le i \le m$

**Asides**

- Write penalty with the $\frac{1}{2}$ in front so that the derivative with respect to $\w$ is $\w$.
- The Regularization Penalty is the same as the Margin Penalty of our prior derivation
- The relative weight between Classification Loss and the Regularization Penalty is $\lambda$
    - The Regularization Penalty is $\lambda$ times more important than Classification Loss
    - Prior derivation: Classification Loss was $C$ times as important as Regularization Penalty
    - so $C = \frac{1}{\lambda}$

# Support Vector Machines: derivation via landmark similarity

We now show another derivation of the Support Vector Machine.

The SVM will apply a particular kind of transformation $\phi$ to $\x$ before fitting a linear model.

## Landmarks and the Similarity transformation

Let us pick a set of distinguished points $\{ \lmk^{(1)}, \lmk^{(2)}, \ldots \}$ in the input domain.

We will refer to these distinguished points as "landmarks" because we will use
them as reference points from which we will measure the similarity (inverse of distance) of each example $\x^\ip$ in the training set.

In particular, let us choose $n'$ landmarks and let
$$
K(\x, \lmk^\ip)
$$
be a measure of similarity between vector $\x$ and the $i^{th}$ landmark.

$K$ will be referred to as a "similarity function" or "kernel".

Then the transformation of $\x^\ip$ into
$$
\hat\x^\ip = [ K(\x^\ip, \lmk^{(1)}), K(\x^\ip, \lmk^{(2)}), \ldots, K(\x^\ip, \lmk^{(n')}),]
$$
is a representation of the original $\x^\ip$ into a "similarity" vector.

The transformed features $\hat{\x}^\ip$ is of length $n'$, each element
representing the distance of $\x^\ip$ to one landmark.

We will do linear classification on these transformed features $\hat\x$ rather than the original $\x$.

The linear classifier creates a hyperplane to separate Positive and Negative examples
by using the dot product to create a score 
$$s(\hat{\x}^\ip) = \hat{\Theta}^T \cdot  \hat{\x}^\ip$$
based on transformed $\hat{\x}^\ip$

This score will determine the prediction.

Score:
$$
s(\hat{\x}^\ip) = \hat{\Theta}^T \cdot \hat{\x}^\ip + b
$$

Score to prediction:
$$
\hat{\y}^\ip = 
\begin{cases}
0 & \text{if } s(\hat{\x}^\ip) < 0 \\
1 & \text{if } s(\hat{\x}^\ip) \ge 0
\end{cases}
$$


**Note** 

- $\hat\Theta$ and $\hat{\x}$ must have the same length
- The $\hat\Theta$ in this derivation is thus **very different** than the $\Theta$ in the original derivation
    - in this derivation the length of $\hat\Theta$ is $n'$ (number of examples) **not** $n$ (number of original features)
    - the elements of this $\hat\Theta$ correspond to *landmarks* **not** features in the original dimensions

So once again, the dot product is doing a form of "pattern matching" of features,
but now 
- we use transformed features: similarity to landmarks
- the pattern is identifying the relative importance of being similar to each landmark

So far, very similar to Logistic Regression: the score determines the prediction.

One difference is that Logistic Regression converts score to  probability
via a sigmoid function:
$$
\hat{p}^\ip = \sigma( s(\x^\ip))
$$
This probability is needed mainly for the Cost function for Logistic Regression (Binary Cross Entropy)
but may be useful as an informative output as well.

We shall soon seen the main difference from Logistic Regression: Hinge Loss
rather than Cross Entropy

## Choosing the landmarks

**TO DO** Picture

How do we choose the landmarks ?  How many do we choose ?

Let's choose each of the $m$ examples in the training set as a landmark so
- $n' = m$
- $\lmk^\ip = \x^\ip$.

This might initially seem to be a large number of landmarks.  

Is it possible to choose fewer ?

Yes, and we will let Machine Learning decide which ones matter!

In [4]:
print("Done")

Done
